In [1]:
# Se importan las librerias
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from Funciones.GEO_LIB import Plot, Poligono, Distancia, Geodecodificacion
from sklearn.feature_extraction.text import TfidfVectorizer

**1. Lectura de datos**

In [2]:
datos=pd.read_csv('Base de datos/datos_lematizados.csv')
datos['lemmatized']=datos['lemmatized'].astype(str)
datos = datos.drop(datos.columns[0], axis=1)

In [3]:
# Filtra el DataFrame aquellas filas que tienen comentarios con espacios vacíos
df_filtered = datos[~datos['lemmatized'].str.contains(r'^\s*$')]
df_filtered.head()

,Unnamed: 0,index,Id,placeId,place name,Categoria query,Year,text,stars,Idioma,traduccion,nps,lemmatized
0,0,2,2,ChIJA7e1akBiF5YRGX2D8me8Pnw,Petrohué Waterfalls,tourist attraction,2023,"Hermoso atractivo turistico, cuanta con sender...",5.0,es,hermoso atractivo turistico cuanta con sendero...,promotor,hermoso atractivo turistico cuantar con sender...
1,1,4,4,ChIJA7e1akBiF5YRGX2D8me8Pnw,Petrohué Waterfalls,tourist attraction,2023,"Muy lindo parque nacional, 100% recomendado, m...",5.0,es,muy lindo parque nacional recomendado muy lind...,promotor,mucho lindo parque nacional recomendado mucho ...
2,2,5,5,ChIJA7e1akBiF5YRGX2D8me8Pnw,Petrohué Waterfalls,tourist attraction,2023,Excelentes paisajes. Un recorrido muy pulcro y...,5.0,es,excelentes paisajes un recorrido muy pulcro y ...,promotor,excelente paisaje uno recorrido mucho pulcro y...
3,3,6,6,ChIJA7e1akBiF5YRGX2D8me8Pnw,Petrohué Waterfalls,tourist attraction,2023,"hermoso, vengo a este lugar desde el año 1968....",4.0,es,hermoso vengo a este lugar desde el ano lament...,promotor,hermoso venir a este lugar desde el ano lament...
4,4,8,8,ChIJA7e1akBiF5YRGX2D8me8Pnw,Petrohué Waterfalls,tourist attraction,2023,"Idiotically crowded, low water levels, mainly ...",1.0,en,estupidamente abarrotado bajos niveles de agua...,detractor,estupidamente abarrotado bajo nivel de agua pr...


**2. Se definen stop words**

In [4]:
# Ejecutar solo una vez la siguiente linea de descarga
#!python -m spacy download es_core_news_sm

# Se definen las stop words que no se van a considerar
nlp = spacy.load("es_core_news_sm")
add_stop = ['ser','tener','deber','haber','debe','lugar','hermoso','precioso','lindo','bello','bonito','vale','pena']
for j in add_stop:
    nlp.Defaults.stop_words.add(j)
esp_stop= list(nlp.Defaults.stop_words)

C:\Users\cmurua\Anaconda3\envs\Turismo\lib\site-packages\spacy\util.py:910: UserWarning:

[W095] Model 'es_core_news_sm' (3.1.0) was trained with spaCy v3.1 and may not be 100% compatible with the current version (3.6.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate



**3. Matriz de frecuencias de palabras por lugar**

In [5]:
# Paso 1: Matriz de frecuencia por review
vect = TfidfVectorizer(stop_words=esp_stop)
vects = vect.fit_transform(datos['lemmatized'])

# Si deseas crear un DataFrame con las características y sus valores TF-IDF
td = pd.DataFrame.sparse.from_spmatrix(vects, columns=vect.get_feature_names_out())


td.columns = vect.get_feature_names_out() #get_feature_names()
td.head()

,aa,aaaa,aaaaa,aaaaaa,aaaaaaa,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa,aaaaaaaaaaaaaaaaaaaaaaiiiiiiiiiiiiiiiiiiiiiii,aaaaaaaaaaaaaaaassasaaasaaaaasaaaaaaaaaaaajfhdjdjsjhdjdhdjvdajysjdudjfywndusbsdhkwydjrudvdhdbdhbdbdbfvdusbydkwhduevsydbwyjwvsudvsubsyduvewjdhsjrhdykwvdkddjshdidhdukshdjhdiebeiekudoehduebuekdjdkwjdjd,aaaaaaahhh,aaaah,...,zunino,zwei,zzz,zzzz,zón,án,ín,ón,ún,ús
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
# Paso 2: Matriz de frecuencia por lugar
td.index = df_filtered['placeId']
td = td.groupby('placeId').sum().sort_values('placeId')
td.head()

,aa,aaaa,aaaaa,aaaaaa,aaaaaaa,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa,aaaaaaaaaaaaaaaaaaaaaaiiiiiiiiiiiiiiiiiiiiiii,aaaaaaaaaaaaaaaassasaaasaaaaasaaaaaaaaaaaajfhdjdjsjhdjdhdjvdajysjdudjfywndusbsdhkwydjrudvdhdbdhbdbdbfvdusbydkwhduevsydbwyjwvsudvsubsyduvewjdhsjrhdykwvdkddjshdidhdukshdjhdiebeiekudoehduebuekdjdkwjdjd,aaaaaaahhh,aaaah,...,zunino,zwei,zzz,zzzz,zón,án,ín,ón,ún,ús
placeId,,,,,,,,,,,,,,,,,,,,,
ChIJ-01C_3EZIpYR9UpdwV9D6Y8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
ChIJ-0drS5c7GJYRycRzXHMYzpU,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
ChIJ-0eSrjUTIpYRFlqkSZzyfF4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
ChIJ-10I8a07GJYRfzQAOYA0vPA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.281771,0.0,0.0,0.0
ChIJ-1zGXfziF5YRJVZTNNO4K1Y,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


**4. Calculo de similaridad**

In [7]:
# Se comparan similitudes entre lugares
coseno_sim = cosine_similarity(td, td)

# Se guarda la matriz resultante
np.save('Resultados/ Matriz_similaridad.npy',coseno_sim)